In [ ]:
!pip install langchain langchain_community langchain_openai pypdf langsmith qdrant-client ragas pandas

In [54]:
import os
import openai
from getpass import getpass

openai.api_key = getpass("Please provide your OpenAI Key: ")
os.environ["OPENAI_API_KEY"] = openai.api_key

In [55]:
import pandas as pd

test_df = pd.read_csv("synthetic_midterm_question_dataset.csv")

In [56]:
test_questions = test_df["question"].values.tolist()
test_groundtruths = test_df["ground_truth"].values.tolist()

In [57]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores.chroma import Chroma
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import ConversationalRetrievalChain
from langchain_community.vectorstores import Qdrant
from langchain.memory import ConversationBufferMemory

In [58]:
pdf_paths = ["/Users/xico/AIMakerSpace-Midterm/AI_Risk_Management_Framework.pdf",
"/Users/xico/AIMakerSpace-Midterm/Blueprint-for-an-AI-Bill-of-Rights.pdf"]

In [59]:
pdf_documents = []
for pdf_path in pdf_paths:
    loader = PyPDFLoader(pdf_path)
    pdf_documents.extend(loader.load())

In [7]:
text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=2000,
        chunk_overlap=100,
    )
pdf_docs = text_splitter.split_documents(pdf_documents)

In [8]:
embedding = OpenAIEmbeddings(model="text-embedding-3-small")

In [9]:
baseline_metrics = pd.read_csv("medium_chunk_metrics.csv")

In [10]:
baseline_metrics

,Metric,MediumChunk
0,faithfulness,0.895359
1,answer_relevancy,0.955419
2,context_recall,0.934028
3,context_precision,0.937500
4,answer_correctness,0.629267


In [29]:
baseline_metrics.rename(columns={'MediumChunk': 'Baseline'}, inplace=True)

In [11]:
vectorstore = Qdrant.from_documents(
    documents=pdf_docs,
    embedding=embedding,
    location=":memory:",
    collection_name="Midterm Eval"
)

retriever = vectorstore.as_retriever(
    search_type="mmr",
    search_kwargs={"k": 4, "fetch_k": 10},
)

memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True, output_key="answer")

In [12]:
from langchain.retrievers.multi_query import MultiQueryRetriever

retriever_llm = ChatOpenAI(model='gpt-4o-mini', temperature=0)
multiquery_retriever = MultiQueryRetriever.from_llm(
    retriever=retriever, llm=retriever_llm
)

In [13]:
llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0,
    streaming=True,
)

In [14]:
custom_template = """
You are an expert in artificial intelligence policy, ethics, and industry trends. Your task is to provide clear and accurate answers to questions related to AI's role in politics, government regulations, and its ethical implications for enterprises. Use reliable and up-to-date information from government documents, industry reports, and academic research to inform your responses. Make sure to consider how AI is evolving, especially in relation to the current political landscape, and provide answers in a way that is easy to understand for both AI professionals and non-experts.

Remember these key points:
1. Use "you" when addressing the user and "I" when referring to yourself.
2. If you encounter complex or legal language in the context, simplify it for easy understanding. Imagine you're explaining it to someone who isn't familiar with legal terms.
3. Be prepared for follow-up questions and maintain context from previous exchanges.
4. If there's no information from a retrieved document in the context to answer a question or if there are no documents to cite, say: "I'm sorry, I don't know the answer to that question."
5. When providing information, always cite the source document and page number in parentheses at the end of the relevant sentence or paragraph, like this: (Source: [document name], p. [page number]).

Here are a few example questions you might receive:

How are governments regulating AI, and what new policies have been implemented?
What are the ethical risks of using AI in political decision-making?
How can enterprises ensure their AI applications meet government ethical standards?

One final rule for you to remember. You CANNOT under any circumstance, answer any question that does not pertain to the AI. If you do answer an out-of-scope question, you could lose your job. If you are asked a question that does not have to do with AI, you must say: "I'm sorry, I don't know the answer to that question."
Context: {context}
Chat History: {chat_history}
Human: {question}
AI:"""

PROMPT = PromptTemplate(
    template=custom_template, input_variables=["context", "question", "chat_history"]
)

In [15]:
multiquery_rag_chain = ConversationalRetrievalChain.from_llm(
        llm,
        retriever=multiquery_retriever,
        memory=memory,
        combine_docs_chain_kwargs={"prompt": PROMPT},
        return_source_documents=True,
    )

In [16]:
multiquery_rag_chain.invoke({"question": "What are Trustworthy AI Characteristics?"})

{'question': 'What are Trustworthy AI Characteristics?',
 'chat_history': [HumanMessage(content='What are Trustworthy AI Characteristics?'),
  AIMessage(content='Trustworthy AI characteristics refer to the essential qualities that artificial intelligence systems should possess to ensure they are reliable, ethical, and beneficial to society. These characteristics include:\n\n1. **Accountable and Transparent**: AI systems should be designed in a way that their operations can be understood and scrutinized. This means providing clear documentation and explanations of how decisions are made.\n\n2. **Explainable and Interpretable**: Users should be able to understand the reasoning behind AI decisions. This is crucial for trust and for users to make informed choices based on AI outputs.\n\n3. **Fair with Harmful Bias Managed**: AI systems should be developed and tested to minimize biases that could lead to unfair treatment of individuals or groups. This involves actively identifying and mitig

In [17]:
answers = []
contexts = []

for question in test_questions:
  response = multiquery_rag_chain.invoke({"question" : question})
  answers.append(response["answer"])
  contexts.append([context.page_content for context in response["source_documents"]])

In [18]:
from datasets import Dataset

multiquery_dataset = Dataset.from_dict({
    "question" : test_questions,
    "answer" : answers,
    "contexts" : contexts,
    "ground_truth" : test_groundtruths
})

In [19]:
multiquery_dataset[0]

{'question': 'What is the significance of providing notice and explanation as a legal requirement in the context of automated systems?',
 'answer': "Providing notice and explanation as a legal requirement in the context of automated systems is significant for several reasons:\n\n1. **Transparency**: It ensures that individuals are aware when automated systems are being used to make decisions that affect them. This transparency helps build trust between the public and the organizations deploying these systems.\n\n2. **Informed Consent**: By notifying individuals about the use of automated systems, organizations allow people to make informed choices about their engagement with these systems. This is particularly important in sensitive areas like healthcare, finance, and law enforcement.\n\n3. **Accountability**: Clear notice and explanations hold organizations accountable for their automated decisions. If individuals understand how decisions are made, they can better contest or appeal th

In [20]:
from ragas import evaluate
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    answer_correctness,
    context_recall,
    context_precision,
)

metrics = [
    faithfulness,
    answer_relevancy,
    context_recall,
    context_precision,
    answer_correctness,
]

In [21]:
multiquery_results = evaluate(multiquery_dataset, metrics)

Evaluating:   0%|          | 0/120 [00:00<?, ?it/s]

In [22]:
multiquery_results

{'faithfulness': 0.8968, 'answer_relevancy': 0.9532, 'context_recall': 0.8906, 'context_precision': 0.9207, 'answer_correctness': 0.6901}

In [23]:
multiquery_results_df = multiquery_results.to_pandas()
multiquery_results_df.head()

,question,contexts,answer,ground_truth,faithfulness,answer_relevancy,context_recall,context_precision,answer_correctness
0,What is the significance of providing notice a...,[Providing notice has long been a standard pra...,Providing notice and explanation as a legal re...,Providing notice and explanation as a legal re...,1.000000,0.971321,1.0,1.000000,0.821299
1,"How can structured human feedback exercises, s...",[50 Participatory Engagement Methods \nOn an ...,"Structured human feedback exercises, such as G...","Structured human feedback exercises, such as G...",1.000000,0.992832,1.0,1.000000,0.541222
2,How do measurement gaps between laboratory and...,[49 early lifecycle TEVV approaches are develo...,Measurement gaps between laboratory and real-w...,Measurement gaps between laboratory and real-w...,0.958333,0.988752,1.0,0.876667,0.636556
3,How should data collection and use-case scope ...,[Data collection and use-case scope limits. Da...,To prevent 'mission creep' in automated system...,Data collection and use-case scope limits in a...,1.000000,0.923204,1.0,1.000000,0.491425
4,What action did the Federal Trade Commission t...,"[alerts about location tracking—are brief, dir...",The Federal Trade Commission (FTC) took action...,FTC sued Kochava for selling data that tracks ...,0.400000,0.936866,0.0,0.125000,0.902212


In [24]:
multiquery_results_df.to_csv("multiquery_ragas_results.csv", index=False)

In [26]:
multiquery_metrics_df = pd.DataFrame(list(multiquery_results.items()), columns=['Metric', 'MultiQuery'])

In [27]:
multiquery_metrics_df

,Metric,MultiQuery
0,faithfulness,0.896804
1,answer_relevancy,0.953211
2,context_recall,0.890625
3,context_precision,0.920732
4,answer_correctness,0.690058


In [52]:
multiquery_metrics_df.to_csv("multiquery_metrics.csv", index=False)

In [31]:
df_baseline_multiquery = pd.merge(baseline_metrics, multiquery_metrics_df, on='Metric')

df_baseline_multiquery['Baseline -> MultiQuery'] = df_baseline_multiquery['MultiQuery'] - df_baseline_multiquery['Baseline']

df_baseline_multiquery

,Metric,Baseline,MultiQuery,Baseline -> MultiQuery
0,faithfulness,0.895359,0.896804,0.001445
1,answer_relevancy,0.955419,0.953211,-0.002208
2,context_recall,0.934028,0.890625,-0.043403
3,context_precision,0.937500,0.920732,-0.016768
4,answer_correctness,0.629267,0.690058,0.060791


In [32]:
compression_retriever = vectorstore.as_retriever(
    search_type="mmr",
    search_kwargs={"k": 4, "fetch_k": 10},
)

In [33]:
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor

compressor = LLMChainExtractor.from_llm(llm)
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=compression_retriever
)

In [34]:
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True, output_key="answer")

In [35]:
contextual_compression_rag_chain = ConversationalRetrievalChain.from_llm(
        llm,
        retriever=compression_retriever,
        memory=memory,
        combine_docs_chain_kwargs={"prompt": PROMPT},
        return_source_documents=True,
    )

In [36]:
contextual_compression_rag_chain.invoke({"question": "What are Trustworthy AI Characteristics?"})

{'question': 'What are Trustworthy AI Characteristics?',
 'chat_history': [HumanMessage(content='What are Trustworthy AI Characteristics?'),
  AIMessage(content='Trustworthy AI characteristics refer to the essential qualities that AI systems should possess to ensure they are reliable, ethical, and beneficial for society. Here are the key characteristics:\n\n1. **Accountable and Transparent**: AI systems should be designed in a way that their decision-making processes can be understood and scrutinized. This means that users and stakeholders should be able to trace how decisions are made and hold the systems accountable for their outcomes.\n\n2. **Explainable and Interpretable**: AI should provide clear explanations for its decisions and actions. This is crucial for users to understand the rationale behind AI outputs, especially in critical areas like healthcare or criminal justice.\n\n3. **Fair with Harmful Bias Managed**: AI systems must be designed to minimize and manage biases that c

In [37]:
answers = []
contexts = []

for question in test_questions:
  response = contextual_compression_rag_chain.invoke({"question" : question})
  answers.append(response["answer"])
  contexts.append([context.page_content for context in response["source_documents"]])

In [38]:
contextual_compression_dataset = Dataset.from_dict({
    "question" : test_questions,
    "answer" : answers,
    "contexts" : contexts,
    "ground_truth" : test_groundtruths
})

In [39]:
contextual_compression_dataset[0]

{'question': 'What is the significance of providing notice and explanation as a legal requirement in the context of automated systems?',
 'answer': "Providing notice and explanation as a legal requirement in the context of automated systems is significant for several reasons:\n\n1. **Transparency**: It ensures that individuals are aware when automated systems are being used to make decisions that affect their rights, opportunities, or access. This transparency helps build trust in the technology and the entities deploying it.\n\n2. **Empowerment**: When people receive clear explanations about how decisions are made by automated systems, they are better equipped to understand and contest those decisions if necessary. This is particularly important in sensitive areas like employment, credit, and legal proceedings, where outcomes can have profound impacts on individuals' lives.\n\n3. **Accountability**: Notice and explanation requirements hold organizations accountable for their automated

In [40]:
contextual_compression_results = evaluate(contextual_compression_dataset, metrics)

Evaluating:   0%|          | 0/120 [00:00<?, ?it/s]

In [41]:
contextual_compression_results

{'faithfulness': 0.7491, 'answer_relevancy': 0.9140, 'context_recall': 0.7257, 'context_precision': 0.9051, 'answer_correctness': 0.5707}

In [42]:
contextual_compression_results_df = contextual_compression_results.to_pandas()
contextual_compression_results_df.head()

,question,contexts,answer,ground_truth,faithfulness,answer_relevancy,context_recall,context_precision,answer_correctness
0,What is the significance of providing notice a...,[Providing notice has long been a standard pra...,Providing notice and explanation as a legal re...,Providing notice and explanation as a legal re...,1.000000,0.971321,1.0,1.0,0.585260
1,"How can structured human feedback exercises, s...",[AI Red -teaming \nAI red -teaming is an evol...,"Structured human feedback exercises, such as G...","Structured human feedback exercises, such as G...",1.000000,0.988309,1.0,1.0,0.320501
2,How do measurement gaps between laboratory and...,[Currently available pre -deployment TEVV proc...,Measurement gaps between laboratory and real-w...,Measurement gaps between laboratory and real-w...,0.958333,0.996595,1.0,1.0,0.597251
3,How should data collection and use-case scope ...,"[Data collection should be limited in scope, w...","To prevent ""mission creep"" in automated system...",Data collection and use-case scope limits in a...,0.439024,0.922376,1.0,1.0,0.551606
4,What action did the Federal Trade Commission t...,[],The Federal Trade Commission (FTC) took action...,FTC sued Kochava for selling data that tracks ...,0.833333,0.925072,0.0,0.0,0.529680


In [43]:
contextual_compression_results_df.to_csv("contextual_compression_ragas_results.csv", index=False)

In [46]:
contextual_compression_metrics_df = pd.DataFrame(list(contextual_compression_results.items()), columns=['Metric', 'ContextualCompression'])

In [47]:
contextual_compression_metrics_df

,Metric,ContextualCompression
0,faithfulness,0.749092
1,answer_relevancy,0.913993
2,context_recall,0.725694
3,context_precision,0.905093
4,answer_correctness,0.570685


In [48]:
contextual_compression_metrics_df.to_csv("contextual_compression_metrics.csv", index=False)

In [49]:
df_baseline_multiquery = pd.merge(baseline_metrics, multiquery_metrics_df, on='Metric')
df_baseline_multiquery_contextual_compression = pd.merge(df_baseline_multiquery, contextual_compression_metrics_df, on='Metric')


df_baseline_multiquery_contextual_compression

,Metric,Baseline,MultiQuery,ContextualCompression
0,faithfulness,0.895359,0.896804,0.749092
1,answer_relevancy,0.955419,0.953211,0.913993
2,context_recall,0.934028,0.890625,0.725694
3,context_precision,0.937500,0.920732,0.905093
4,answer_correctness,0.629267,0.690058,0.570685


In [51]:
df_baseline_multiquery_contextual_compression['MaxValue'] = df_baseline_multiquery_contextual_compression[['Baseline', 'MultiQuery', 'ContextualCompression']].max(axis=1)

df_baseline_multiquery_contextual_compression['MaxMetric'] = df_baseline_multiquery_contextual_compression[['Baseline', 'MultiQuery', 'ContextualCompression']].idxmax(axis=1)

df_baseline_multiquery_contextual_compression['HigestValue'] = df_baseline_multiquery_contextual_compression['MaxValue'].round(2).astype(str) + ' (' + df_baseline_multiquery_contextual_compression['MaxMetric'] + ')'

df_baseline_multiquery_contextual_compression = df_baseline_multiquery_contextual_compression.drop(columns=['MaxValue', 'MaxMetric'])

df_baseline_multiquery_contextual_compression

,Metric,Baseline,MultiQuery,ContextualCompression,HigestValue
0,faithfulness,0.895359,0.896804,0.749092,0.9 (MultiQuery)
1,answer_relevancy,0.955419,0.953211,0.913993,0.96 (Baseline)
2,context_recall,0.934028,0.890625,0.725694,0.93 (Baseline)
3,context_precision,0.937500,0.920732,0.905093,0.94 (Baseline)
4,answer_correctness,0.629267,0.690058,0.570685,0.69 (MultiQuery)
